In [1]:
def get_flow_latency_webget(n=100):
    import pandas as pd
    import pickle
    all_flows_data_file = "webget_2016.csv"
    flow_data = pd.read_csv(all_flows_data_file).groupby(["src_ip","dst_ip", "sport","dport"])["latency"].agg(list)
    sent_flow_data_file = "sent_flows_webget_2016.pickle"
    flows = pickle.load(open(sent_flow_data_file,"rb"))
    flow_latency = {}
    for key, value_list in flow_data.items():
        if tuple(key) in flows.keys():
            flow_latency[tuple(key)] = value_list[:n]
    return flow_latency

In [2]:
def get_flow_latency_seattle(n=100):
    import pandas as pd
    flow_data = pd.read_csv("seattle_data.csv")
    flow_latency = {}
    for column  in flow_data.columns:
        src = column.split("->")[0]
        dst = column.split("->")[1]
        key_tuple = (src,dst,1234,5678)
        column_data = flow_data[column].values[:n]
        flow_latency[key_tuple] = (column_data).tolist()
    return flow_latency

In [494]:
def get_flow_latency_synthetic(n=100):
    import pandas as pd
    flow_data = pd.read_csv("synthetic_data.csv")
    flow_latency = {}
    for column  in flow_data.columns:
        src = column.split("->")[0]
        dst = column.split("->")[1]
        key_tuple = (src,dst,1234,5678)
        column_data = flow_data[column].values[:n]
        flow_latency[key_tuple] = (column_data).tolist()
    return flow_latency

In [503]:
flow_latency = get_flow_latency_webget(100)
# flow_latency = get_flow_latency_seattle(100)
# flow_latency = get_flow_latency_synthetic(100)

In [504]:
flow_latency.keys()

dict_keys([('1.1.1.1', '10.0.10.1', 1234, 5678), ('1.1.1.1', '10.0.10.2', 1234, 5678), ('1.1.1.1', '10.0.10.3', 1234, 5678), ('108.177.96.103', '10.0.10.1', 1234, 5678), ('108.177.96.104', '10.0.10.1', 1234, 5678), ('108.177.96.105', '10.0.10.1', 1234, 5678), ('108.177.96.106', '10.0.10.1', 1234, 5678), ('108.177.96.147', '10.0.10.1', 1234, 5678), ('108.177.96.99', '10.0.10.1', 1234, 5678), ('131.203.2.15', '10.0.10.1', 1234, 5678), ('131.203.2.15', '10.0.10.2', 1234, 5678), ('131.203.2.19', '10.0.10.1', 1234, 5678), ('131.203.2.19', '10.0.10.2', 1234, 5678), ('131.203.2.23', '10.0.10.1', 1234, 5678), ('131.203.2.23', '10.0.10.2', 1234, 5678), ('131.203.2.27', '10.0.10.1', 1234, 5678), ('131.203.2.27', '10.0.10.2', 1234, 5678), ('131.203.2.29', '10.0.10.1', 1234, 5678), ('131.203.2.29', '10.0.10.2', 1234, 5678), ('131.203.2.30', '10.0.10.1', 1234, 5678), ('131.203.2.30', '10.0.10.2', 1234, 5678), ('131.203.2.34', '10.0.10.1', 1234, 5678), ('131.203.2.34', '10.0.10.2', 1234, 5678), ('13

In [505]:
# 从log文件中直接读取estimate quantile的结果
import json
f = open("./log/hosts/h2-eth0_new.json")
lines = f.readlines()
exp_quantile = {}
estimate_flow_count = {}
for line in lines:
    # print(line)
    line = eval(line)
    flow = tuple([line["src_ip"],line['dst_ip'],line['sport'],line['dport']])
    lat = line["latency"]
    quantile = [int(line["quantile0"])]
    count = [int(line["count"])]
    if flow not in exp_quantile.keys():
        exp_quantile[flow] = []
        estimate_flow_count[flow] = []
    # line = line.split(",")[0].split(":")
    # lat = line[1]
    exp_quantile[flow].append(quantile)
    estimate_flow_count[flow].append(count)
import numpy as np
print(len(flow_latency))
estimate_quantile_from_file = {}
for flow, value in flow_latency.items():
    if flow not in exp_quantile.keys():
        break
    sort_value = sorted(value[:100])
    estimate_quantile_from_file[flow] = exp_quantile[flow][-1]
    # print(sort_value,int(0.9*len(sort_value)))
    # print(flow, sort_value[int(0.9*len(sort_value)) ], exp_quantile[flow][-1], estimate_flow_count[flow][-1])

60


In [506]:
import pickle
with open("log/hosts/sent_flow_quantile.pickle", "rb") as f:
    measured_flow_quantile = pickle.load(f)

In [507]:
estimate_quantile_from_file

{('1.1.1.1', '10.0.10.1', 1234, 5678): [470553],
 ('1.1.1.1', '10.0.10.2', 1234, 5678): [77179],
 ('1.1.1.1', '10.0.10.3', 1234, 5678): [1618255],
 ('108.177.96.103', '10.0.10.1', 1234, 5678): [1099806],
 ('108.177.96.104', '10.0.10.1', 1234, 5678): [1438516],
 ('108.177.96.105', '10.0.10.1', 1234, 5678): [1161421],
 ('108.177.96.106', '10.0.10.1', 1234, 5678): [1076332],
 ('108.177.96.147', '10.0.10.1', 1234, 5678): [1194055],
 ('108.177.96.99', '10.0.10.1', 1234, 5678): [1281535],
 ('131.203.2.15', '10.0.10.1', 1234, 5678): [2714256],
 ('131.203.2.15', '10.0.10.2', 1234, 5678): [3398234],
 ('131.203.2.19', '10.0.10.1', 1234, 5678): [2014340],
 ('131.203.2.19', '10.0.10.2', 1234, 5678): [2587947],
 ('131.203.2.23', '10.0.10.1', 1234, 5678): [2232460],
 ('131.203.2.23', '10.0.10.2', 1234, 5678): [2401614],
 ('131.203.2.27', '10.0.10.1', 1234, 5678): [2577403],
 ('131.203.2.27', '10.0.10.2', 1234, 5678): [2712858],
 ('131.203.2.29', '10.0.10.1', 1234, 5678): [2749716],
 ('131.203.2.29',

In [508]:
QUANTILE_PHI = 0.3
for flow, latency in flow_latency.items():
    sorted_latency = sorted(latency)
    if flow not in measured_flow_quantile.keys():
        break
    # print(sorted_latency)
    print("flow:{}, sorted_flow_latency:{}".format(flow, sorted_latency))
    print("flow:{}, real quantile:{},  estimate_quantile_from_file:{}, estimate_flow_counts:{}, measured quantile:{}, measured flow count:{}".format(\
        flow, sorted_latency[int(len(sorted_latency) * QUANTILE_PHI) + 1],  (estimate_quantile_from_file[flow]), estimate_flow_count[flow][-1], (measured_flow_quantile[flow][0]),(measured_flow_quantile[flow][1]),))

flow:('1.1.1.1', '10.0.10.1', 1234, 5678), sorted_flow_latency:[4191, 4192, 5202, 23741, 23862, 25067, 25215, 25354, 25584, 25960, 26544, 26557, 26989, 27248, 27346, 27362, 27718, 28010, 28115, 28200, 28209, 28353, 29421, 30328, 31765, 32000, 32070, 32352, 32369, 32390, 32467, 32522, 32675, 33216, 33242, 33280, 33668, 33669, 33768, 33865, 33906, 34052, 34231, 34671, 34825, 35084, 37605, 38349, 40199, 40418, 42360, 43533, 43648, 45639, 53123, 55878, 56537, 56864, 57950, 57979, 60218, 62529, 62894, 63879, 65069, 65724, 66081, 67540, 67708, 70598, 72038, 72787, 102398, 134477, 211434, 247581, 262182, 263352, 301294, 308274, 323671, 324221, 343373, 362448, 399675, 488803, 549917, 585805, 636875, 777238, 864899, 990685, 1285869, 1508507, 1658101, 1676105, 2055025, 2898203, 7652702, 10011288]
flow:('1.1.1.1', '10.0.10.1', 1234, 5678), real quantile:32522,  estimate_quantile_from_file:[470553], estimate_flow_counts:[99], measured quantile:[470553, 470553, 470553], measured flow count:[99, 99,

In [509]:
from math import log
def ALE(measured_quantile, flow_latency, QUANTILE_PHI):
    ale = 0
    print(len(flow_latency))
    count = 0
    
    for flow, latency in flow_latency.items():
        if flow not in measured_quantile.keys():
            break
        sorted_latency = sorted(latency)
        real_quantile = sorted_latency[int(len(sorted_latency) * QUANTILE_PHI)]
        if len(measured_quantile[flow]) == 2:
            estimate_quantile = min(measured_quantile[flow][0])
        else:
            estimate_quantile = min(measured_quantile[flow])
        log_real_quantile = log(real_quantile,2)
        log_estimate_quantile = log(estimate_quantile,2)
        # print("--------",flow, estimate_quantile, real_quantile, abs(log_real_quantile-log_estimate_quantile), measured_quantile[flow][0],measured_quantile[flow][1])
        if abs(log_real_quantile-log_estimate_quantile) > 2.8:
            print("~~~~~~~~~~~~~~~~~~~~~",flow, estimate_quantile, real_quantile, abs(log_real_quantile-log_estimate_quantile), measured_quantile[flow][0],measured_quantile[flow][1])
            # continue
        print(flow, abs(log_real_quantile-log_estimate_quantile))
        ale = ale + abs(log_real_quantile-log_estimate_quantile)
        count += 1
    return ale/count

In [510]:
# 读取出来的
print(ALE(measured_flow_quantile, flow_latency, QUANTILE_PHI = 0.9))

60
('1.1.1.1', '10.0.10.1', 1234, 5678) 0.8781744417030701
('1.1.1.1', '10.0.10.2', 1234, 5678) 0.014314730074875825
~~~~~~~~~~~~~~~~~~~~~ ('1.1.1.1', '10.0.10.3', 1234, 5678) 1618255 71382 4.502734827449736 [1618255, 1618255, 1618255] [99, 99, 99]
('1.1.1.1', '10.0.10.3', 1234, 5678) 4.502734827449736
('108.177.96.103', '10.0.10.1', 1234, 5678) 0.1315745488461353
('108.177.96.104', '10.0.10.1', 1234, 5678) 0.2537150880007353
('108.177.96.105', '10.0.10.1', 1234, 5678) 0.04320787598177844
('108.177.96.106', '10.0.10.1', 1234, 5678) 0.13351317931866902
('108.177.96.147', '10.0.10.1', 1234, 5678) 0.01200891927742731
('108.177.96.99', '10.0.10.1', 1234, 5678) 0.06584859787766817
('131.203.2.15', '10.0.10.1', 1234, 5678) 0.18954947936817135
('131.203.2.15', '10.0.10.2', 1234, 5678) 0.09439641474496341
('131.203.2.19', '10.0.10.1', 1234, 5678) 0.44422930579646547
('131.203.2.19', '10.0.10.2', 1234, 5678) 0.2007328828906978
('131.203.2.23', '10.0.10.1', 1234, 5678) 0.19294595000015846
('131.

In [243]:
# 读取出来的
print(ALE(measured_flow_quantile, flow_latency, QUANTILE_PHI = 0.4))

82
('172.85.216.33', '153.71.233.185', 1234, 5678) 1.1098040090047423
('185.150.215.110', '153.71.233.185', 1234, 5678) 1.3726769975551996
('185.150.215.110', '83.238.77.63', 1234, 5678) 2.173829455898228
('157.152.30.13', '153.71.233.185', 1234, 5678) 1.735236780249454
('157.152.30.13', '83.238.77.63', 1234, 5678) 1.9999999999999991
('195.151.62.163', '153.71.233.185', 1234, 5678) 1.8565855137604537
('58.134.63.128', '153.71.233.185', 1234, 5678) 1.4907800616644487
('255.142.80.71', '153.71.233.185', 1234, 5678) 0.3560020289818908
('255.142.80.71', '201.76.203.143', 1234, 5678) 2.2439255828860896
('60.28.184.150', '153.71.233.185', 1234, 5678) 0.20700037091946566
('249.23.191.154', '153.71.233.185', 1234, 5678) 0.7369655941662057
('179.99.199.196', '153.71.233.185', 1234, 5678) 1.970853654340484
('247.182.242.46', '153.71.233.185', 1234, 5678) 1.4662162232044054
('123.77.179.237', '153.71.233.185', 1234, 5678) 1.4410327078117886
('186.134.173.241', '153.71.233.185', 1234, 5678) 2.3249

In [236]:
# 存在文件中的
print(ALE(estimate_quantile_from_file, flow_latency, QUANTILE_PHI = 0.4))

82
('172.85.216.33', '153.71.233.185', 1234, 5678) 1.5486206536574354
('185.150.215.110', '153.71.233.185', 1234, 5678) 0.6201519291834572
('185.150.215.110', '83.238.77.63', 1234, 5678) 0.8519013610108672
('157.152.30.13', '153.71.233.185', 1234, 5678) 0.6832645743329611
('157.152.30.13', '83.238.77.63', 1234, 5678) 1.481126689736615
('195.151.62.163', '153.71.233.185', 1234, 5678) 0.11211036632153704
('58.134.63.128', '153.71.233.185', 1234, 5678) 0.4715694576986138
('255.142.80.71', '153.71.233.185', 1234, 5678) 0.24679376482658455
('255.142.80.71', '201.76.203.143', 1234, 5678) 0.27563444261342784
('60.28.184.150', '153.71.233.185', 1234, 5678) 1.2725258548110912
('249.23.191.154', '153.71.233.185', 1234, 5678) 0.76836079044174
('179.99.199.196', '153.71.233.185', 1234, 5678) 0.9871187087032673
('247.182.242.46', '153.71.233.185', 1234, 5678) 0.25857475764309257
('123.77.179.237', '153.71.233.185', 1234, 5678) 0.0534392589614594
('186.134.173.241', '153.71.233.185', 1234, 5678) 1.4

IndexError: list index out of range